In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [2]:
USER = 'postgres'
PASSWORD = 'password'
HOST = 'localhost'
PORT = '5433'
DATABASE = 'tradinganalytics_db'

#Initialize Database Connection
host = HOST
username = USER
password = PASSWORD
port = PORT
database = DATABASE

conn = psycopg2.connect(
    host = host,
    database = database,
    user = username,
    port = port,
    password = password
)

def query(query,connection):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except Exception as e:
        print("Error executing query:", e)
        return None

In [5]:
query_string = "select * from dm_tradinganalytics_sch.view_ml_training_data_5m"

result = query(query_string,conn)

df = pd.DataFrame(result,columns=["close","high","low","datetime","ticker","open_close_diff","high_low_diff","volume","direction_volume","cumulative_volume"])

df.set_index('datetime',inplace=True)

In [6]:
df.head()

,close,high,low,ticker,open_close_diff,high_low_diff,volume,direction_volume,cumulative_volume
datetime,,,,,,,,,
2024-02-23 09:30:00,182.7100067138672,183.8000030517578,181.60000610351562,AMD,0.85000610351564,2.19999694824218,4170507,4170507,4170507
2024-02-23 09:35:00,181.88999938964844,183.47000122070312,181.82000732421875,AMD,-0.83599853515626,1.64999389648437,2286954,-2286954,1883553
2024-02-23 09:40:00,181.8699951171875,182.6999053955078,181.8300018310547,AMD,-0.02000427246094,0.8699035644531,1540312,-1540312,343241
2024-02-23 09:45:00,180.02999877929688,181.97999572753906,179.77999877929688,AMD,-1.89999389648437,2.19999694824218,2515930,-2515930,-2172689
2024-02-23 09:50:00,179.8300018310547,180.47999572753906,179.39999389648438,AMD,-0.27499389648436,1.08000183105468,1667770,-1667770,-3840459


In [17]:
import tensorflow as tf

In [18]:
del df['ticker']

In [19]:
df.head()

,transaction_cnt,close,high,low,open_close_diff,high_low_diff,volume,direction_volume,cumulative_volume
datetime,,,,,,,,,
2023-01-09 09:30:00,19,64.9,64.9,64.82,0.08,0.08,1601,1601,12748
2023-01-09 09:40:00,26,64.9,64.91,64.81,0.09,0.10,2661,2661,15409
2023-01-09 10:30:00,25,64.7,64.72,64.7,-0.02,0.02,645,-645,14807
2023-01-09 10:35:00,27,64.56,64.6,64.56,-0.04,0.04,1502,-1502,13305
2023-01-09 10:40:00,1,64.55,64.55,64.55,0.00,0.00,100,100,13405


In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler(feature_range=(0,1))

In [20]:
len(df)

1302